In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install -U sentence-transformers

In [7]:
import random
import os
import re
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'
SUBMISSION_PATH = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

SEED = 42
reset_seeds(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
test = pd.read_csv(f'{DATA_PATH}test.csv')
train_ft = pd.read_csv(f'{DATA_PATH}category_train_120000_ver4.csv')
train_ft.head()
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [9]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.17.0-py3-none-any.whl size=34845034 sha256=f1c965d02ac5234beac1d7489c14573d0a19ee6b6017d009b3a3a87ffb1f68a9
  Stored in directory: /root/.cache/pip/wheels/90/bb/ae/15e1cf26d17183040bac268fbf8d1ca1f45f7f4daa0fb76b91
Successfully built kiwipiepy-model


In [10]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.tokenize(train_ft['question'][0])

[Token(form='면', tag='NNG', start=0, len=1),
 Token(form='지', tag='VV', start=1, len=1),
 Token(form='ᆫ', tag='ETM', start=1, len=1),
 Token(form='장치', tag='NNG', start=2, len=2),
 Token(form='가', tag='JKS', start=4, len=1),
 Token(form='뭐', tag='NP', start=6, len=1),
 Token(form='이', tag='VCP', start=7, len=0),
 Token(form='야', tag='EF', start=7, len=1),
 Token(form='?', tag='SF', start=8, len=1)]

In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

def get_kiwi_nouns(text):
    pos_result = kiwi.tokenize(text)
    nouns = [word for word, pos, _,_ in pos_result if pos.startswith('N')]
    return ' '.join(nouns)

def kiwi_evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절

    input_embedding = embedding_model.encode(get_kiwi_nouns(input_text))
    generated_embedding = embedding_model.encode(get_kiwi_nouns(generated_text))

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [12]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [13]:
from sklearn.metrics.pairwise import pairwise_distances
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

def cosine_score(input_text, generated_text):
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    cosine_similarity_score = cosine_similarity(input_embedding, generated_embedding)
    cosine_similarity_score = max(cosine_similarity_score, 0)

    return cosine_similarity_score

def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [14]:
from transformers import GPT2LMHeadModel

model_name = 'skt/kogpt2-base-v2'
finetune_model = 'kogpt2_category_1_0310120000_5epoch_0.14518loss'


model = GPT2LMHeadModel.from_pretrained(f'{DATA_PATH}model/{finetune_model}').to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [15]:
def post_processing(text):
    import re
    end = text.rfind('</s>')
    if end:
        text = text[:end]
    # end = texts[i].rfind('.')
    # texts[i] = texts[i][:end+1]
    text = text.strip()
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    return text

In [ ]:
# @torch.no_grad()
# def Chatbot(max_length,
#             df=test, model=model, tokenizer=tokenizer, device=device, num_samples=5,
#             temperature=1.0, top_k=50, top_p=1.0):
#     reset_seeds(SEED)
#     model.eval()
#     preds = []
#     scores_lst = []
#     for i in tqdm(range(df.shape[0])):
#         pred = None
#         best_similarity_score = -1
#         score_lst = []
#         for _ in range(num_samples):
#             text = df['질문'][i].strip().replace('"', '')
#             input_text = "<q>" + text + "</s><a>"
#             input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
#             result_ids = model.generate(input_ids=input_ids.to(device),
#                                         max_length=max_length,
#                                         temperature=temperature,
#                                         top_k=top_k,
#                                         top_p=top_p,
#                                         do_sample=True,
#                                         num_return_sequences=1
#                                         )

#             q_len = len(input_text)
#             generated_text = tokenizer.decode(result_ids[0])
#             generated_text = generated_text[q_len+1:]

#             similarity_score = evaluate_similarity(text, generated_text)
#             score_lst.append(similarity_score)
#             if similarity_score > best_similarity_score:
#                 best_similarity_score = similarity_score
#                 pred = generated_text
#         preds.append(pred)
#         scores_lst.append(score_lst)
#     return preds, scores_lst

In [16]:
def Chatbot(input_text, model=model, tokenizer=tokenizer, device=device,
            max_length=200, temperature=0.87, top_k=27, top_p=0.7, num_samples=5, generated=True):

    model.eval()
    input_text = input_text.strip().replace('"', '')
    text = '<q>' + input_text + '</s><a>'
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    score_lst = []
    generated_texts = []
    for i in range(num_samples):
        result_ids = model.generate(input_ids,
                                    max_length=max_length,
                                    temperature=temperature,
                                    top_k=top_k,
                                    top_p=top_p,
                                    do_sample=True,
                                    num_return_sequences=1,
                                    )

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = post_processing(generated_text[q_len:])

        similarity_score = evaluate_similarity(input_text, generated_text)
        score_lst.append(similarity_score)
        generated_texts.append((similarity_score,generated_text))
        if generated:
            print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    return best_generated_text, score_lst

In [17]:
def Chatbot(input_text, model=model, tokenizer=tokenizer, device=device,
            max_length=200, temperature=0.87, top_k=27, top_p=0.7, num_samples=5, generated=True):

    model.eval()
    input_text = input_text.strip().replace('"', '')
    text = '<q>' + input_text + '</s><a>'
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    score_lst = []
    generated_texts = []
    for i in range(num_samples):
        result_ids = model.generate(input_ids,
                                    max_length=max_length,
                                    temperature=temperature,
                                    top_k=top_k,
                                    top_p=top_p,
                                    do_sample=True,
                                    num_return_sequences=1,
                                    )

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = post_processing(generated_text[q_len:])

        similarity_score = evaluate_similarity(input_text, generated_text)
        score_lst.append(similarity_score)
        generated_texts.append((similarity_score,generated_text))
        if generated:
            print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    return best_generated_text, score_lst

In [18]:
def chatbot(input_text, model=model, tokenizer=tokenizer, device=device,
            max_length=200, temperature=0.87, top_k=27, top_p=0.7):
    model.eval()
    input_text = input_text.strip().replace('"', '')
    text = '<q>' + input_text + '</s><a>'
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    result_ids = model.generate(input_ids,
                                max_length=max_length,
                                temperature=temperature,
                                top_k=top_k,
                                top_p=top_p,
                                do_sample=True,
                                num_return_sequences=5
                                )

    generated_text = tokenizer.decode(result_ids[0])
    generated_text = generated_text[q_len:]

    return generated_text

In [19]:
Chatbot('인테리어 잘할 수 있는 팁을 알려줘', generated=False, max_length=200)

('인테리어에서 벽지를 선택할 때 고려해야 할 중요한 요소는 다양합니다. 먼저, 공간의 크기, 조명 상황, 가구의 색상과 장식 요소들을 고려해야 합니다. 또한, 벽지의 패턴과 질감도 전체적인 디자인에 맞게 조화롭게 선택하는 것이 중요합니다. 또한, 벽지를 선택할 때는 벽지의 재질과 내구성, 청소 및 유지보수의 용이성도 고려해야 합니다. 수지 벽지는 습기에 강하고 청결하나, 튼튼함이 부족합니다. 반면, 천벽지는 내구성이 있지만 수지 벽지만큼 깔끔한 재질이 필요합니다. 따라서, 벽지를 선택할 때는 이러한 다양한 요소들을 종합적으로 고려하여야 합니다.',
 [0.17149099707603455,
  0.17051416933536528,
  0.13777036666870116,
  0.17112044692039488,
  0.15563451774915057])

In [20]:
Chatbot('인테리어 잘할 수 있는 팁을 알려줘', generated=False, max_length=200, temperature=0.5, top_k=20, top_p=0.95)

('인테리어에서 벽지를 선택할 때 고려해야 할 중요한 요소는 다양합니다. 먼저, 공간의 크기, 조명 상황, 가구의 색상과 장식 요소들을 고려해야 합니다. 또한, 벽지의 패턴과 질감도 전체적인 디자인에 맞게 조화롭게 선택하는 것이 중요합니다. 또한, 벽지의 재질과 내구성, 청소 유지 및 보관의 편의성을 고려하는 것도 중요합니다. 이러한 다양한 측면을 종합적으로 고려하여 적합한 벽지를 선택하는 것이 좋습니다.',
 [0.15565253576716861,
  0.14096393372331345,
  0.15565253576716861,
  0.17189699411392212,
  0.15565253576716861])

In [21]:
Chatbot('새집증후군이 생긴거 같은데 어떻게 해야해? 그리고 도배 마감 잘하는 법 좀', generated=False, max_length=200)

('새집증후군이를 제거하고 싶다면 천연 소재, 포름알데히드 처리를 하지 않은 자재를 사용해야하며 창문을 열어 자연스럽게 환기를 시켜줘야 합니다. 또한, 새집으로 이사하는 경우 입주하기 전 7일 정도 전에 실내 난방온도를 30도 이상으로 8시간 정도 유지하여 휘발성 유기물질을 제거해야 합니다. 그리고 도배 마감은 내구성, 내수성, 내마모성이 우수하지만 가격이 비싸며 충격 흡수율이 낮아 노약자가 있는 집안에는 적절하지 않다는 단점이 있습니다.',
 [0.21699484144487688,
  0.25034593353195794,
  0.22593325582043877,
  0.22593325582043877,
  0.22593325582043877])

### CAT VER1 - 5ep (temperature=0.87, top_k=27, top_p=0.7)

In [22]:
mean_score_cat1_ep5 = []
answer_list_cat1_ep5 = []
for text in tqdm(test['질문'].tolist()) :
    reset_seeds(SEED)
    answer, score = Chatbot(text, generated=False, max_length=200)
    answer_list_cat1_ep5.append(answer)
    mean_score_cat1_ep5.append(np.mean(score))

  0%|          | 0/130 [00:00<?, ?it/s]

In [23]:
import pickle
with open(f'{DATA_PATH}answer_list_cat1_ep5.pkl', 'wb') as f:
    pickle.dump(answer_list_cat1_ep5, f)

with open(f'{DATA_PATH}mean_score_cat1_ep5.pkl', 'wb') as f:
    pickle.dump(mean_score_cat1_ep5, f)

In [24]:
for i in range(len(mean_score_cat1_ep5)) :
    print(f'질문 : {test["질문"].tolist()[i]}')
    print(f'답변 : {mean_score_cat1_ep5[i]}')
    print(' ')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변 : 0.1919466406871111
 
질문 : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
답변 : 0.4491465525160875
 
질문 : 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
답변 : 0.5118719997613327
 
질문 : 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
답변 : 0.3806439564742294
 
질문 : 도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
답변 : 0.33100036563948976
 
질문 : 철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?
답변 : 0.5311736150582631
 
질문 : 어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?
답변 : 0.5308038795536215
 
질문 : 도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 무엇인가요?
답변 : 0.24897092633370682
 
질문 : 도배지에 얼룩이 생기는 다양한 원인들에 대해서 자세히 알려주세요.
답변 : 0.2532072702458255
 
질문 : 고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?
답변 : 0.561116575772696
 
질문 : 유성페인트의 환경 오염에 대한 예시를 알려주세요.
답변 : 0.20023751810034324
 
질문 : 훼손과 오염은 어떻게 다른가요? 또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?
답변 : 0.5408968435365327
 
질문 : 철골콘크리트 구조에서 사용되는 철근의 재료적 특징은 무엇인가요?"
답변 : 0.5413341726336562
 
질문 : 방염벽지가 실크벽지와 합지벽지와

### CAT VER1 - 5ep (temperature=0.5, top_k=20, top_p=0.95)

In [25]:
mean_score_cat1_ep5_topk20 = []
answer_list_cat1_ep5_topk20 = []
for text in tqdm(test['질문'].tolist()) :
    reset_seeds(SEED)
    answer, score = Chatbot(text, generated=False, max_length=200, temperature=0.5, top_k=20, top_p=0.95)
    answer_list_cat1_ep5_topk20.append(answer)
    mean_score_cat1_ep5_topk20.append(np.mean(score))

  0%|          | 0/130 [00:00<?, ?it/s]

In [26]:
import pickle
with open(f'{DATA_PATH}answer_list_cat1_ep5_topk20.pkl', 'wb') as f:
    pickle.dump(answer_list_cat1_ep5_topk20, f)

with open(f'{DATA_PATH}mean_score_cat1_ep5_topk20.pkl', 'wb') as f:
    pickle.dump(mean_score_cat1_ep5_topk20, f)

In [27]:
for i in range(len(answer_list_cat1_ep5_topk20)) :
    print(f'질문 : {test["질문"].tolist()[i]}')
    print(f'답변 : {answer_list_cat1_ep5_topk20[i]}')
    print(' ')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변 : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
 
질문 : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
답변 : 주로 녹 오염으로 인해 도배지에 붉은색의 녹이 생기는 현상이 발생합니다. 이러한 현상은 다음과 같은 원인들로 인해 발생할 수 있습니다. 첫째, 높은 습도로 인한 금속 손상. 높은 습기로 인해 도배지 안쪽의 금속이 녹을 수 있으며, 이로 인해 녹이 도배지에 베어나올 수 있습니다. 습기를 관리하기 위해서는 제습기나 환기를 통해 실내 습도를 조절하고, 전문가의 도움을 받아 보수 작업을 해야 합니다. 둘째, 누수로 인한 곰팡이 발생. 누수로 인해 도배지가 젖은 상태가 계속되면 곰팡이가 생길 수 있습니다. 이 경우에는 누수를 제거하고 곰팡이가 발생한 부분을 교체하거나 전체를 보수 작업을 함으로써 해결할 수 있습니다. 이러한 작업은 전문가의 도움이 필요합니다. 그러므로, 이 문제를 해결하기 위해서는 건물의 소유자나 거주자가 적절한 대응을 취해야 하며, 전문가와 협력하는 것이 좋습니다.
 
질문 : 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
답변 : 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.
 
질문 : 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
답변 : 일반적으로 철골 구조는 

### CAT VER1 - 6ep (temperature=0.87, top_k=27, top_p=0.7)

In [28]:
from transformers import GPT2LMHeadModel

model_name = 'skt/kogpt2-base-v2'
finetune_model = 'kogpt2_category_1_0310120000_6epoch_0.13981loss'



model = GPT2LMHeadModel.from_pretrained(f'{DATA_PATH}model/{finetune_model}').to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                                    bos_token='</s>',
                                                    eos_token='</s>',
                                                    unk_token='<unk>',
                                                    pad_token='<pad>',
                                                    mask_token='<mask>')

In [29]:
mean_score_cat1_ep6 = []
answer_list_cat1_ep6 = []
for text in tqdm(test['질문'].tolist()) :
    reset_seeds(SEED)
    answer, score = Chatbot(text, generated=False, max_length=200)
    answer_list_cat1_ep6.append(answer)
    mean_score_cat1_ep6.append(np.mean(score))

  0%|          | 0/130 [00:00<?, ?it/s]

In [30]:
import pickle
with open(f'{DATA_PATH}answer_list_cat1_ep6.pkl', 'wb') as f:
    pickle.dump(answer_list_cat1_ep6, f)

with open(f'{DATA_PATH}mean_score_cat1_ep6.pkl', 'wb') as f:
    pickle.dump(mean_score_cat1_ep6, f)

In [31]:
for i in range(len(answer_list_cat1_ep6)) :
    print(f'질문 : {test["질문"].tolist()[i]}')
    print(f'답변 : {answer_list_cat1_ep6[i]}')
    print(' ')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변 : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
 
질문 : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
답변 : 주로 녹 오염으로 인해 도배지에 붉은색의 녹이 생기는 현상이 발생합니다. 이러한 현상은 다음과 같은 원인들로 인해 발생할 수 있습니다. 첫째, 높은 습도로 인한 금속 손상. 높은 습기로 인해 도배지 안쪽의 금속이 녹을 수 있으며, 이로 인해 녹이 도배지에 베어나올 수 있습니다. 습기를 관리하기 위해서는 제습기나 환기를 통해 실내 습도를 조절하고, 전문가의 도움을 받아 보수 작업을 해야 합니다. 둘째, 누수로 인한 곰팡이 발생. 누수로 인해 도배지가 젖은 상태가 계속되면 곰팡이가 생길 수 있습니다. 이 경우에는 누수를 제거하고 곰팡이가 발생한 부분을 교체하거나 전체를 보수 작업을 함으로써 해결할 수 있습니다. 이러한 작업은 전문가의 도움이 필요합니다. 그러므로, 이 문제를 해결하기 위해서는 건물의 소유자나 거주자가 적절한 대응을 취해야 하며, 전문가와 협력하는 것이 좋습니다.
 
질문 : 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
답변 : 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.
 
질문 : 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
답변 : 일반적으로, 철골 구조는

### CAT VER1 - 6ep (temperature=0.5, top_k=20, top_p=0.95)

In [32]:
mean_score_cat1_ep6_topk20 = []
answer_list_cat1_ep6_topk20 = []
for text in tqdm(test['질문'].tolist()) :
    reset_seeds(SEED)
    answer, score = Chatbot(text, generated=False, max_length=200, temperature=0.5, top_k=20, top_p=0.95)
    answer_list_cat1_ep6_topk20.append(answer)
    mean_score_cat1_ep6_topk20.append(np.mean(score))

  0%|          | 0/130 [00:00<?, ?it/s]

In [33]:
import pickle
with open(f'{DATA_PATH}answer_list_cat1_ep6_topk20.pkl', 'wb') as f:
    pickle.dump(answer_list_cat1_ep6_topk20, f)

with open(f'{DATA_PATH}mean_score_cat1_ep6_topk20.pkl', 'wb') as f:
    pickle.dump(mean_score_cat1_ep6_topk20, f)

In [34]:
for i in range(len(answer_list_cat1_ep6_topk20)) :
    print(f'질문 : {test["질문"].tolist()[i]}')
    print(f'답변 : {answer_list_cat1_ep6_topk20[i]}')
    print(' ')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변 : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
 
질문 : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
답변 : 주로 녹 오염으로 인해 도배지에 붉은색의 녹이 생기는 현상이 발생합니다. 이러한 현상은 다음과 같은 원인들로 인해 발생할 수 있습니다. 첫째, 높은 습도로 인한 금속 손상. 높은 습기로 인해 도배지 안쪽의 금속이 녹을 수 있으며, 이로 인해 녹이 도배지에 베어나올 수 있습니다. 습기를 관리하기 위해서는 제습기나 환기를 통해 실내 습도를 조절하고, 전문가의 도움을 받아 보수 작업을 해야 합니다. 둘째, 누수로 인한 곰팡이 발생. 누수로 인해 도배지가 젖은 상태가 계속되면 곰팡이가 생길 수 있습니다. 이 경우에는 누수를 제거하고 곰팡이가 발생한 부분을 교체하거나 전체를 보수 작업을 함으로써 해결할 수 있습니다. 이러한 작업은 전문가의 도움이 필요합니다. 그러므로, 이 문제를 해결하기 위해서는 건물의 소유자나 거주자가 적절한 대응을 취해야 하며, 전문가와 협력하는 것이 좋습니다.
 
질문 : 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
답변 : 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.
 
질문 : 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
답변 : 일반적으로 철골 구조는 


# ***3ep, 4ep, 5ep, 6ep 비교


In [39]:
with open(f'{DATA_PATH}answer_list_cat1_ep3.pkl', 'rb') as f:
    answer_list_cat1_ep3 = pickle.load(f)
with open(f'{DATA_PATH}answer_list_cat1_ep3_topk20.pkl', 'rb') as f:
    answer_list_cat1_ep3_topk20 = pickle.load(f)

with open(f'{DATA_PATH}answer_list_cat1_ep4.pkl', 'rb') as f:
    answer_list_cat1_ep4 = pickle.load(f)
with open(f'{DATA_PATH}answer_list_cat1_ep4_topk20.pkl', 'rb') as f:
    answer_list_cat1_ep4_topk20 = pickle.load(f)

In [41]:
for i in range(len(test)) :
    print(f'질문 : {test["질문"].tolist()[i]}')
    print()
    print(f'> 답변 3ep topk27: {answer_list_cat1_ep3[i]}')
    print(f'> 답변 3ep topk20: {answer_list_cat1_ep3_topk20[i]}')
    print(f'> 답변 4ep topk27: {answer_list_cat1_ep4[i]}')
    print(f'> 답변 4ep topk20: {answer_list_cat1_ep4_topk20[i]}')
    print(' ')
    print(f'> 답변 5ep topk27: {answer_list_cat1_ep5[i]}')
    print(f'> 답변 5ep topk20: {answer_list_cat1_ep5_topk20[i]}')
    print(f'> 답변 6ep topk27: {answer_list_cat1_ep6[i]}')
    print(f'> 답변 6ep topk20: {answer_list_cat1_ep6_topk20[i]}')
    print(' ')
    print('#######################')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?

> 답변 3ep topk27: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
> 답변 3ep topk20: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
> 답변 4ep topk27: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
> 답변 4ep topk20: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 경우 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
 
> 답변 5ep topk27: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
> 답변 5ep topk20: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트

In [ ]:
DATA_PATH

'/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [ ]:
# test_6ep_noparam.to_csv(f'test_6ep_noparam.csv', index=False, encoding='utf-8-sig')

In [ ]:
# answer_list = post_processing(answer_list)
# answer_list
for a, b, c in zip(test['질문'].tolist(), answer_list_5ep, mean_score) :
    print(f'질문 : {a}')
    print(f'답변 : {b}')
    print(f'답변 : {c}')
    print(' ')

질문 : 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
답변 : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
답변 : [0.1919466406871111, 0.1919466406871111, 0.1919466406871111, 0.1919466406871111, 0.1919466406871111]
 
질문 : 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
답변 : 높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 도배지에 발생할 수 있습니다. 따라서 높은 습도로 인해 도배지 내부의 금속 물질이 도배지 표면에 스며들면 효과적인 보수 및 제거가 필요합니다. 따라서 높은 습도가 지속되는 환경에서는 도배작업 시 습도 높은 온도를 사용하는 것이 중요하며, 도배지의 금속 물질이 녹는 것을 방지하기 위해서는 적절한 방법으로 보수 및 재단장을 해야합니다.
답변 : [0.26172347522917244, 0.14623316824436186, 0.2580958456565172, 0.35528990138660776, 0.37734221017360686]
 
질문 : 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
답변 : 큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.
답변 : [0.5118719997613327, 0.5118719997613327, 0.5118719997613327, 0.51187199976

In [ ]:
vector_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
pred_vector = vector_model.encode(answer_list_5ep)
pred_vector.shape

(130, 512)

In [ ]:
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
submission.iloc[:,1:] = pred_vector
submission

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.026558,-0.020150,0.015123,0.017831,0.064715,0.018982,0.002021,-0.006165,-0.006314,...,-0.013723,-0.000593,-0.031992,-0.039757,0.001512,0.022232,0.030489,0.040009,0.001398,0.013257
1,TEST_001,0.003926,0.020359,0.005585,0.002899,0.083786,-0.003711,-0.012671,-0.023206,0.001001,...,-0.021571,-0.000138,0.052596,-0.030294,-0.026886,0.051963,-0.043098,-0.025048,0.013413,0.017596
2,TEST_002,0.027461,-0.047922,-0.024391,-0.000196,0.122556,-0.054846,-0.028347,-0.026099,0.043037,...,-0.034606,-0.017103,0.026653,-0.023691,0.022718,0.013030,-0.042122,-0.038602,-0.048148,0.092198
3,TEST_003,0.042312,0.024629,-0.023008,0.045299,0.115577,-0.051378,-0.033345,0.028962,0.023886,...,-0.074070,-0.006998,0.055625,-0.009289,-0.033153,0.007479,-0.010673,-0.034691,0.031323,0.052541
4,TEST_004,0.020813,0.018052,-0.005413,0.005860,0.125650,-0.013503,0.057134,0.052642,-0.043287,...,-0.030031,-0.008895,0.033195,-0.004471,-0.036152,0.010436,-0.010917,0.006888,-0.004987,0.055993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.029159,-0.044267,0.028296,0.017269,0.118264,-0.067284,0.066199,0.045351,0.050197,...,0.013426,0.009427,0.049926,-0.040957,0.015311,0.051693,0.042989,-0.026956,-0.029772,0.007569
126,TEST_126,-0.011318,0.024199,-0.010428,0.034779,0.096595,-0.020263,-0.022025,-0.046513,-0.027042,...,-0.036890,-0.006135,-0.001099,-0.038049,-0.029020,0.046892,-0.023480,0.050811,-0.027023,0.007681
127,TEST_127,0.014216,0.055457,-0.022567,0.033716,0.100355,0.003071,0.003213,0.009190,-0.038783,...,-0.056672,-0.065226,-0.004017,-0.026441,-0.014779,0.038532,0.002473,0.025062,-0.022473,0.029961
128,TEST_128,0.076996,-0.004353,-0.060054,0.032989,0.107366,-0.037963,0.003364,0.054680,0.009391,...,-0.010608,0.006609,0.021276,-0.016115,-0.053511,-0.028914,-0.034626,0.008319,0.033348,-0.002455


In [ ]:
submission.to_csv(f'{SUBMISSION_PATH}kogpt_132220_6ep_rule_based.csv', index=False)
submission.to_csv(f'submission_kogpt_132220_6ep_rule_based.csv', index=False)


In [ ]:
submission.to_csv(f'submission_kogpt_132220_5ep_rule_based.csv', index=False)
